<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Market-Data/blob/main/Recent_Trades_List_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 近期成交列表 (Recent Trades List)
# GET /openApi/swap/v2/quote/trades
# 不需要 API 金鑰簽名

In [ ]:
import time
import requests
import hmac
from hashlib import sha256

In [ ]:
APIURL = "https://open-api.bingx.com"  # API 基本網址
APIKEY = ""  # 使用者的 API 金鑰，需要填入有效金鑰
SECRETKEY = ""  # 使用者的秘密金鑰，需要填入有效密鑰

In [ ]:
def demo():
    payload = {}  # GET 請求的請求體通常為空
    path = '/openApi/swap/v2/quote/trades'  # API 路徑：取得近期成交列表
    method = "GET"  # HTTP 請求方法

    paramsMap = {
        "symbol": "BTC-USDT",  # 交易對符號，必須包含連字號，如 BTC-USDT
        "limit": 10  # 查詢限制，回傳最近 10 筆成交

    }

    paramsStr = parseParam(paramsMap)  # 將參數字典轉成 URL 查詢字串，並加上 timestamp
    return send_request(method, path, paramsStr, payload)  # 發送請求並回傳結果

In [ ]:
def get_sign(api_secret, payload):
    # 用 HMAC-SHA256 演算法，對參數字串做簽名

    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    print("sign=" + signature)  # 輸出簽名以供調試
    return signature  # 回傳簽名字串

In [ ]:
def send_request(method, path, urlpa, payload):

    # 組合完整請求 URL，包含簽名參數
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    print(url)  # 輸出完整 URL，方便除錯

    headers = {
        'X-BX-APIKEY': APIKEY,  # 在 HTTP 標頭帶入 API 金鑰
    }

    # 發送 HTTP 請求，包含方法、URL、標頭與請求體
    response = requests.request(method, url, headers=headers, data=payload)
    return response.text  # 回傳 API 回應內容（字串）

In [ ]:
def parseParam(paramsMap):
    sortedKeys = sorted(paramsMap)  # 參數鍵排序，保證簽名一致性

    # 將參數組成 URL 查詢字串，例如 key1=value1&key2=value2
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])

    if paramsStr != "":
        # 若有參數，加入 timestamp 時間戳（毫秒）
        return paramsStr+"&timestamp="+str(int(time.time() * 1000))
    else:
        # 若無參數，只有 timestamp
        return paramsStr+"timestamp="+str(int(time.time() * 1000))

In [ ]:
if __name__ == '__main__':
    print("demo:", demo())  # 執行示範函式並印出結果